In [1]:
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Flatten
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard

In [3]:
file = open("nietzsche.txt", encoding="utf-8")
text_niet = file.read()
# 732020:732176

file_date = open("baddate.txt", encoding="utf-8")
text_date = file_date.read()

EPOCHS = 10
diversity = 0.5

In [13]:
niet = text_niet.split("\n")
date = text_date.split("\n")
text = []

i = 0
j = 0
counter = 0
while i < len(niet) and j < len(date):
    if counter % 2 == 0:
        text.append(niet[i])
        i += 1
    else:
        text.append(date[j])
        j += 1
    counter += 1

# text = text + niet[i:len(niet)]
# text = text + date[j:len(date)]

sep = "\n"
text = sep.join(text)

In [14]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1

model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
        
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

C:\Users\timjh\AppData\Local\Temp\ipykernel_48580\315039847.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
C:\Users\timjh\AppData\Local\Temp\ipykernel_48580\315039847.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), seqlen, len(chars)), dtype=n

Epoch 1/10
57/58 [============================>.] - ETA: 0s - loss: 3.1732 - categorical_crossentropy: 3.1732 - accuracy: 0.1914
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "hat *
106.
bad, is he? *

LEIGH
How is i"
hat *
106.
bad, is he? *

LEIGH
How is ing the the the the the the the the thet and there the thes the the the the the the theng the the the the the the the theng the the the the the the the thesthe the the there the the theng the thengete the theng and tout and the the the the the the the the the thent and the the the the the the there the the the the the the the the the theng the the the the the there the the thent ang the the the the
----- diversity: 0.5
----- Generating with seed: "hat *
106.
bad, is he? *

LEIGH
How is i"
hat *
106.
bad, is he? *

LEIGH
How is itt tod tort ton thents cof wof  ontongon and thantithateent west terent the pand ane the wits contead and pes thered wite thes, thendithestite the the me thertas thest w